In [6]:
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.local import LocalSession

#No need to be signed in (eg.g. SSO) when using LocalSession local code mode
sagemaker_session = LocalSession()
sagemaker_session.config = {'local': {'local_code': True}}
DUMMY_IAM_ROLE = 'arn:aws:iam::999999999999:role/dummy-role/AmazonSageMaker-ExecutionRole-20000101T000000'

train = "file://local/data"
validation = "file://local/data"
output = "file:///tmp/sagemaker-xgboost-local"

hyperparams = {
    "eta": 0.1,
    "num_class": 30,
    "num_round": 20,
    "objective": "multi:softprob",
    "hite_rate_k": 5
}

estimator = XGBoost(
    entry_point="train.py",
    hyperparameters=hyperparams,
    role=DUMMY_IAM_ROLE,
    instance_count=1,
    instance_type="local",
    framework_version="1.3-1",
    output_path=output,
    sagemaker_session=sagemaker_session
)

estimator.fit(
    inputs = {
        "train": train,
        "validation": validation
    }
)

# since the train.py also includes the code to load the model during inference
# we can deploy the estimator locally immediatelly, it knows where the model.tar.gz exists.
predictor = estimator.deploy(
    initial_instance_count=1, 
    instance_type='local'
)

payload = '1, 0.455, 0.365, 0.095, 0.514, 0.2245, 0.101, 0.15'
preds = predictor.predict(payload)

print(f"prediction: {preds}")


Creating tgm742578s-algo-1-w3g9k ... 
Creating tgm742578s-algo-1-w3g9k ... done
Attaching to tgm742578s-algo-1-w3g9k
tgm742578s-algo-1-w3g9k | [2021-12-22 22:59:46.752 1aef23de8043:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
tgm742578s-algo-1-w3g9k | [2021-12-22:22:59:46:INFO] Imported framework sagemaker_xgboost_container.training
tgm742578s-algo-1-w3g9k | [2021-12-22:22:59:46:INFO] No GPUs detected (normal if no gpus installed)
tgm742578s-algo-1-w3g9k | [2021-12-22:22:59:46:INFO] Invoking user training script.
tgm742578s-algo-1-w3g9k | [2021-12-22:22:59:46:INFO] Installing module with the following command:
tgm742578s-algo-1-w3g9k | /miniconda3/bin/python3 -m pip install . 
tgm742578s-algo-1-w3g9k | Processing /opt/ml/code
tgm742578s-algo-1-w3g9k |   Preparing metadata (setup.py) ... done
tgm742578s-algo-1-w3g9k | Building wheels for collected packages: train
tgm742578s-algo-1-w3g9k |   Building wheel for train (setup.py) ... done
tgm742578s-algo-1-w3g9k |   Created wheel

Exception in thread Thread-16:
Traceback (most recent call last):
  File "/Users/jonajoha/miniforge3/envs/sagemaker_sdk_py38/lib/python3.8/site-packages/sagemaker/local/image.py", line 837, in run
    _stream_output(self.process)
  File "/Users/jonajoha/miniforge3/envs/sagemaker_sdk_py38/lib/python3.8/site-packages/sagemaker/local/image.py", line 899, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jonajoha/miniforge3/envs/sagemaker_sdk_py38/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Users/jonajoha/miniforge3/envs/sagemaker_sdk_py38/lib/python3.8/site-packages/sagemaker/local/image.py", line 842, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/private/var/folders/bq/yl58k5y566l70xyqp7hh5j8h0000gn/T/tmppnue6ou